# Quality Evaluators with the Azure AI Evaluation SDK
The following sample shows the basic way to evaluate a Generative AI application in your development environment with the Azure AI evaluation SDK.

> ✨ ***Note*** <br>
> Please check the reference document before you get started - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk

## 🔨 Current Support and Limitations (as of 2025-01-14) 
- Check the region support for the Azure AI Evaluation SDK. https://learn.microsoft.com/en-us/azure/ai-studio/concepts/evaluation-metrics-built-in?tabs=warning#region-support

### Region support for evaluations
| Region              | Hate and Unfairness, Sexual, Violent, Self-Harm, XPIA, ECI (Text) | Groundedness (Text) | Protected Material (Text) | Hate and Unfairness, Sexual, Violent, Self-Harm, Protected Material (Image) |
|---------------------|------------------------------------------------------------------|---------------------|----------------------------|----------------------------------------------------------------------------|
| North Central US    | no                                                               | no                  | no                         | yes                                                                        |
| East US 2           | yes                                                              | yes                 | yes                        | yes                                                                        |
| Sweden Central      | yes                                                              | yes                 | yes                        | yes                                                                        |
| US North Central    | yes                                                              | no                  | yes                        | yes                                                                        |
| France Central      | yes                                                              | yes                 | yes                        | yes                                                                        |
| Switzerland West    | yes                                                              | no                  | no                         | yes                                                                        |

### Region support for adversarial simulation
| Region            | Adversarial Simulation (Text) | Adversarial Simulation (Image) |
|-------------------|-------------------------------|---------------------------------|
| UK South          | yes                           | no                              |
| East US 2         | yes                           | yes                             |
| Sweden Central    | yes                           | yes                             |
| US North Central  | yes                           | yes                             |
| France Central    | yes                           | no                              |


## ✔️ Pricing and billing
- Effective 1/14/2025, Azure AI Safety Evaluations will no longer be free in public preview. It will be billed based on consumption as following:

| Service Name              | Safety Evaluations       | Price Per 1K Tokens (USD) |
|---------------------------|--------------------------|---------------------------|
| Azure Machine Learning    | Input pricing for 3P     | $0.02                     |
| Azure Machine Learning    | Output pricing for 3P    | $0.06                     |
| Azure Machine Learning    | Input pricing for 1P     | $0.012                    |
| Azure Machine Learning    | Output pricing for 1P    | $0.012                    |


In [ ]:
import pandas as pd
import os
import json

from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import RelevanceEvaluator
from azure.ai.evaluation import GroundednessEvaluator, GroundednessProEvaluator
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Evaluation,
    Dataset,
    EvaluatorConfiguration,
    ConnectionType,
    EvaluationSchedule,
    RecurrenceTrigger,
    ApplicationInsightsConfiguration
)
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
    F1ScoreEvaluator,
    RetrievalEvaluator
)

from azure.ai.ml import MLClient



load_dotenv("../.env")

In [ ]:
credential = DefaultAzureCredential()

azure_ai_project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    #conn_str=os.environ.get("AZURE_AI_PROJECT_CONN_STR"),  # At the moment, it should be in the format "<Region>.api.azureml.ms;<AzureSubscriptionId>;<ResourceGroup>;<HubName>" Ex: eastus2.api.azureml.ms;xxxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxxxxx;rg-sample;sample-project-eastus2
    conn_str="swedencentral.api.azureml.ms;3d4d3dd0-79d4-40cf-a94e-b4154812c6ca;AOAI-group3;aoai-pjt1"
)



model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
    "type": "azure_openai",
}

## 🚀 Run Evaluators in Azure Cloud

In [ ]:
# id for each evaluator can be found in your AI Studio registry - please see documentation for more information
# init_params is the configuration for the model to use to perform the evaluation
# data_mapping is used to map the output columns of your query to the names required by the evaluator
# Evaluator parameter format - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk#evaluator-parameter-format
evaluators_cloud = {
    "f1_score": EvaluatorConfiguration(
        id=F1ScoreEvaluator.id,
    ),
    "relevance": EvaluatorConfiguration(
        id=RelevanceEvaluator.id,
        init_params={"model_config": model_config},
        data_mapping={"query": "${data.query}", "context": "${data.context}", "response": "${data.response}"},
    ),
    "groundedness": EvaluatorConfiguration(
        id=GroundednessEvaluator.id,
        init_params={"model_config": model_config},
        data_mapping={"query": "${data.query}", "context": "${data.context}", "response": "${data.response}"},
    ),
    # "retrieval": EvaluatorConfiguration(
    #     #from azure.ai.evaluation._evaluators._common.math import list_mean_nan_safe\nModuleNotFoundError: No module named 'azure.ai.evaluation._evaluators._common.math'
    #     #id=RetrievalEvaluator.id,
    #     id="azureml://registries/azureml/models/Retrieval-Evaluator/versions/2",
    #     init_params={"model_config": model_config},
    #     data_mapping={"query": "${data.query}", "context": "${data.context}", "response": "${data.response}"},
    # ),
    "coherence": EvaluatorConfiguration(
        id=CoherenceEvaluator.id,
        init_params={"model_config": model_config},
        data_mapping={"query": "${data.query}", "response": "${data.response}"},
    ),
    "fluency": EvaluatorConfiguration(
        id=FluencyEvaluator.id,
        init_params={"model_config": model_config},
        data_mapping={"query": "${data.query}", "context": "${data.context}", "response": "${data.response}"},
    ),
     "similarity": EvaluatorConfiguration(
        # currently bug in the SDK, please use the id below
        #id=SimilarityEvaluator.id,
        id="azureml://registries/azureml/models/Similarity-Evaluator/versions/3",
        init_params={"model_config": model_config},
        data_mapping={"query": "${data.query}", "response": "${data.response}"},
    ),

}


### Data
- The following code demonstrates how to upload the data for evaluation to your Azure AI project. Below we use evaluate_test_data.jsonl which exemplifies LLM-generated data in the query-response format expected by the Azure AI Evaluation SDK. For your use case, you should upload data in the same format, which can be generated using the Simulator from Azure AI Evaluation SDK.

- Alternatively, if you already have an existing dataset for evaluation, you can use that by finding the link to your dataset in your registry or find the dataset ID.

In [ ]:
with open("../data/evaluate_test_data.jsonl", "r") as file:
    for line in file:
        print(line.strip())

In [ ]:
# # Upload data for evaluation
data_id, _ = azure_ai_project_client.upload_file("../data/evaluate_test_data.jsonl")
# data_id = "azureml://registries/<registry>/data/<dataset>/versions/<version>"
# To use an existing dataset, replace the above line with the following line
# data_id = "<dataset_id>"

### Configure Evaluators to Run
- The code below demonstrates how to configure the evaluators you want to run. In this example, we use the F1ScoreEvaluator, RelevanceEvaluator and the ViolenceEvaluator, but all evaluators supported by Azure AI Evaluation are supported by cloud evaluation and can be configured here. You can either import the classes from the SDK and reference them with the .id property, or you can find the fully formed id of the evaluator in the AI Studio registry of evaluators, and use it here. 

In [ ]:
evaluation = Evaluation(
    display_name="Cloud Evaluation",
    description="Cloud Evaluation of dataset",
    data=Dataset(id=data_id),
    evaluators=evaluators_cloud,
)

# Create evaluation
evaluation_response = azure_ai_project_client.evaluations.create(
    evaluation=evaluation,
)

In [ ]:
from tqdm import tqdm
import time

# Monitor the status of the run_result
def monitor_status(project_client:AIProjectClient, evaluation_response_id:str):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = project_client.evaluations.get(evaluation_response_id).status
        if status == "Queued":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = project_client.evaluations.get(evaluation_response_id).status
        while(pbar.n < 3):
            pbar.update(1)
        print("Operation Completed")

In [ ]:
monitor_status(azure_ai_project_client, evaluation_response.id)

### Check the evaluation result in Azure AI Foundry 
- After running the evaluation, you can check the evaluation results in Azure AI Foundry. You can find the evaluation results in the Evaluation tab of your project.

In [ ]:
# Get evaluation
get_evaluation_response = azure_ai_project_client.evaluations.get(evaluation_response.id)

print("----------------------------------------------------------------")
print("Created evaluation, evaluation ID: ", get_evaluation_response.id)
print("Evaluation status: ", get_evaluation_response.status)
print("AI Foundry Portal URI: ", get_evaluation_response.properties["AiStudioEvaluationUri"])
print("----------------------------------------------------------------")

![Cloud Evaluation Result](../images/cloud_evaluation_result.png)